# Import The Packages

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio

In [2]:
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

In [3]:
import matplotlib.pyplot as plt

In [4]:
from patchify import patchify

In [5]:
from gammaTransformation import transform_img

# Utils Function

In [6]:
def load_data(path):
    """ X = Images and Y = masks """

    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

In [7]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [41]:
def augment_data(images, masks, save_path):

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting names """
        name = x.split("\\")[-1].split(".")[0]
        

        """ Reading image and mask """
        src = cv2.imread(x, cv2.IMREAD_COLOR)
        bgr = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
        x = transform_img(bgr)
        
        y = imageio.mimread(y)[0]

        top, bottom = [28] * 2
        left, right = [37,38]
        
        x = cv2.copyMakeBorder(x, top, bottom, left, right, cv2.BORDER_CONSTANT, value= 0)
        y = cv2.copyMakeBorder(y, top, bottom, left, right, cv2.BORDER_CONSTANT, value= 0)
        
    
        aug = HorizontalFlip(p=1.0)
        augmented = aug(image=x, gt=y)
        x1 = augmented["image"]
        y1 = augmented["gt"]

        aug = VerticalFlip(p=1.0)
        augmented = aug(image=x, gt=y)
        x2 = augmented["image"]
        y2 = augmented["gt"]

        aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
        augmented = aug(image=x, gt=y)
        x3 = augmented['image']
        y3 = augmented['gt']

        aug = GridDistortion(p=1)
        augmented = aug(image=x, gt=y)
        x4 = augmented['image']
        y4 = augmented['gt']

        aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
        augmented = aug(image=x, gt=y)
        x5 = augmented['image']
        y5 = augmented['gt']

        X = [x, x1, x2, x3, x4, x5]
        Y = [y, y1, y2, y3, y4, y5]


        index = 0
        for i, m in zip(X, Y):


            patches_img = patchify(x, (128, 128,3), step=128)
            patches_gt = patchify(y, (128, 128), step=128)
        
        
            for i in range(patches_img.shape[0]):
                for j in range(patches_img.shape[1]):
                    if i == 0:
                        if j == 0 or j == 4:
                            continue
                    elif i == 4:
                        if j == 0 or j == 4:
                            continue
                    
                    single_patch_img = patches_img[i,j,:,:][0]
                    tmp_image_name = f"{name}_{index}_{i}{j}.jpg" 
                    
                    single_patch_gt = patches_gt[i,j,:,:]
                    tmp_gt_name = f"{name}_{index}_{i}{j}.jpg"
                    
                    if idx % 4 == 0:
                        fol = "valid\\"
                    else:
                        fol = "train\\"
                        
                    image_path = os.path.join(save_path,fol,"image", tmp_image_name)
                    print(image_path)
                    cv2.imwrite(image_path, single_patch_img)
                    gt_path = os.path.join(save_path, fol, "gt", tmp_gt_name)
                    cv2.imwrite(gt_path, single_patch_gt)
                    
    
                    
            index += 1

In [42]:
""" Seeding """
np.random.seed(42)

""" Load the data """
data_path = "D:\\Data sets\\DRIVE"
(train_x, train_y), (test_x, test_y) = load_data(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")


Train: 20 - 20
Test: 20 - 20


In [45]:
""" Creating directories """
create_dir("D:\\gamma_data_128\\train\\image")
create_dir("D:\\gamma_data_128\\train\\gt")
create_dir("D:\\gamma_data_128\\valid\\image")
create_dir("D:\\gamma_data_128\\valid\\gt")

In [46]:
augment_data(train_x, train_y, "D:\\gamma_data_128\\")

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

D:\gamma_data_128\valid\image\21_training_0_01.jpg
D:\gamma_data_128\valid\image\21_training_0_02.jpg
D:\gamma_data_128\valid\image\21_training_0_03.jpg
D:\gamma_data_128\valid\image\21_training_0_10.jpg
D:\gamma_data_128\valid\image\21_training_0_11.jpg
D:\gamma_data_128\valid\image\21_training_0_12.jpg
D:\gamma_data_128\valid\image\21_training_0_13.jpg
D:\gamma_data_128\valid\image\21_training_0_14.jpg
D:\gamma_data_128\valid\image\21_training_0_20.jpg
D:\gamma_data_128\valid\image\21_training_0_21.jpg
D:\gamma_data_128\valid\image\21_training_0_22.jpg
D:\gamma_data_128\valid\image\21_training_0_23.jpg
D:\gamma_data_128\valid\image\21_training_0_24.jpg
D:\gamma_data_128\valid\image\21_training_0_30.jpg
D:\gamma_data_128\valid\image\21_training_0_31.jpg
D:\gamma_data_128\valid\image\21_training_0_32.jpg
D:\gamma_data_128\valid\image\21_training_0_33.jpg
D:\gamma_data_128\valid\image\21_training_0_34.jpg
D:\gamma_data_128\valid\image\21_training_0_41.jpg
D:\gamma_data_128\valid\image\2

  5%|████▏                                                                              | 1/20 [00:00<00:08,  2.28it/s]

D:\gamma_data_128\valid\image\21_training_2_13.jpg
D:\gamma_data_128\valid\image\21_training_2_14.jpg
D:\gamma_data_128\valid\image\21_training_2_20.jpg
D:\gamma_data_128\valid\image\21_training_2_21.jpg
D:\gamma_data_128\valid\image\21_training_2_22.jpg
D:\gamma_data_128\valid\image\21_training_2_23.jpg
D:\gamma_data_128\valid\image\21_training_2_24.jpg
D:\gamma_data_128\valid\image\21_training_2_30.jpg
D:\gamma_data_128\valid\image\21_training_2_31.jpg
D:\gamma_data_128\valid\image\21_training_2_32.jpg
D:\gamma_data_128\valid\image\21_training_2_33.jpg
D:\gamma_data_128\valid\image\21_training_2_34.jpg
D:\gamma_data_128\valid\image\21_training_2_41.jpg
D:\gamma_data_128\valid\image\21_training_2_42.jpg
D:\gamma_data_128\valid\image\21_training_2_43.jpg
D:\gamma_data_128\valid\image\21_training_3_01.jpg
D:\gamma_data_128\valid\image\21_training_3_02.jpg
D:\gamma_data_128\valid\image\21_training_3_03.jpg
D:\gamma_data_128\valid\image\21_training_3_10.jpg
D:\gamma_data_128\valid\image\2

 10%|████████▎                                                                          | 2/20 [00:00<00:07,  2.55it/s]

...

D:\gamma_data_128\train\image\38_training_0_02.jpg
D:\gamma_data_128\train\image\38_training_0_03.jpg
D:\gamma_data_128\train\image\38_training_0_10.jpg
D:\gamma_data_128\train\image\38_training_0_11.jpg
D:\gamma_data_128\train\image\38_training_0_12.jpg
D:\gamma_data_128\train\image\38_training_0_13.jpg
D:\gamma_data_128\train\image\38_training_0_14.jpg
D:\gamma_data_128\train\image\38_training_0_20.jpg
D:\gamma_data_128\train\image\38_training_0_21.jpg
D:\gamma_data_128\train\image\38_training_0_22.jpg
D:\gamma_data_128\train\image\38_training_0_23.jpg
D:\gamma_data_128\train\image\38_training_0_24.jpg
D:\gamma_data_128\train\image\38_training_0_30.jpg
D:\gamma_data_128\train\image\38_training_0_31.jpg
D:\gamma_data_128\train\image\38_training_0_32.jpg
D:\gamma_data_128\train\image\38_training_0_33.jpg
D:\gamma_data_128\train\image\38_training_0_34.jpg
D:\gamma_data_128\train\image\38_training_0_41.jpg
D:\gamma_data_128\train\image\38_training_0_42.jpg
D:\gamma_data_128\train\image\3

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:11<00:01,  1.25s/it]

D:\gamma_data_128\train\image\39_training_0_01.jpg
D:\gamma_data_128\train\image\39_training_0_02.jpg
D:\gamma_data_128\train\image\39_training_0_03.jpg
D:\gamma_data_128\train\image\39_training_0_10.jpg
D:\gamma_data_128\train\image\39_training_0_11.jpg
D:\gamma_data_128\train\image\39_training_0_12.jpg
D:\gamma_data_128\train\image\39_training_0_13.jpg
D:\gamma_data_128\train\image\39_training_0_14.jpg
D:\gamma_data_128\train\image\39_training_0_20.jpg
D:\gamma_data_128\train\image\39_training_0_21.jpg
D:\gamma_data_128\train\image\39_training_0_22.jpg
D:\gamma_data_128\train\image\39_training_0_23.jpg
D:\gamma_data_128\train\image\39_training_0_24.jpg
D:\gamma_data_128\train\image\39_training_0_30.jpg
D:\gamma_data_128\train\image\39_training_0_31.jpg
D:\gamma_data_128\train\image\39_training_0_32.jpg
D:\gamma_data_128\train\image\39_training_0_33.jpg
D:\gamma_data_128\train\image\39_training_0_34.jpg
D:\gamma_data_128\train\image\39_training_0_41.jpg
D:\gamma_data_128\train\image\3

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.73it/s]

D:\gamma_data_128\train\image\40_training_0_01.jpg
D:\gamma_data_128\train\image\40_training_0_02.jpg
D:\gamma_data_128\train\image\40_training_0_03.jpg
D:\gamma_data_128\train\image\40_training_0_10.jpg
D:\gamma_data_128\train\image\40_training_0_11.jpg
D:\gamma_data_128\train\image\40_training_0_12.jpg
D:\gamma_data_128\train\image\40_training_0_13.jpg
D:\gamma_data_128\train\image\40_training_0_14.jpg
D:\gamma_data_128\train\image\40_training_0_20.jpg
D:\gamma_data_128\train\image\40_training_0_21.jpg
D:\gamma_data_128\train\image\40_training_0_22.jpg
D:\gamma_data_128\train\image\40_training_0_23.jpg
D:\gamma_data_128\train\image\40_training_0_24.jpg
D:\gamma_data_128\train\image\40_training_0_30.jpg
D:\gamma_data_128\train\image\40_training_0_31.jpg
D:\gamma_data_128\train\image\40_training_0_32.jpg
D:\gamma_data_128\train\image\40_training_0_33.jpg
D:\gamma_data_128\train\image\40_training_0_34.jpg
D:\gamma_data_128\train\image\40_training_0_41.jpg
D:\gamma_data_128\train\image\4

In [42]:
### Shift 16 images from train to valid

In [39]:
# augment_data(test_x, test_y, "new_data/test/", augment=False)

In [62]:
set([4,4])

{4}